## COM6012 - 2017: Practical Quiz 3

This quiz is built upon your previous quiz 2 so you can reuse the PracticalQuiz2Solution. You will still use logistic regression to perform classification in the [spam](./files/spambase.data) dataset. You will split the samples in two datasets, one for training (with 70% of the samples), and one for testing (with 30% of the samples). You will use a regularisation parameter of 0.01 and an elastic net parameter of 0.1.

Step 1 [2 marks]: Use PCA to reduce the features to keep (approximately) 95% of the total variance using computePrincipalComponentsAndExplainedVariance. Report the reduced feature dimension and variance kept (in percentage), as well as the classification accuracy.

Step 2 [1 mark]: Use PCA to reduce the features to keep (approximately) 85% of the total variance using computeSVD. Report the reduced feature dimension and variance kept (in percentage), as well as the classification accuracy.

Step 3 [2 marks]: Please tell which original attribute (feature, i.e., raw feature before applying PCA) explains the most variance in the data and how much variance (in percentage) it has explained.

Provide your answers in the Notebook.

## Answers:
1)Accuracy: 0.6997105643994211

reduced feature dimensiton: 2 pc's

variance: 99.80699856872896

2)Accuracy: 0.683206106870229

reduced feature dimensiton: 2 pc's

variance: 99.80699856872896

3)variance summary: [0.09324324068491728,1.6655843234835528,0.2541600473337551,1.946447346588171,0.45227342485092964,0.07497962843616227,0.15322633420900933,0.16085830998932185,0.0776267998072253,0.4157095251221379,
0.040620251608818554,0.742524255407614,0.09062255504001963,0.11234819972973736,0.06699993229260176,0.6819319336533649,0.19718513519745243,0.28209102715855694,3.1523315909584833,0.25986228107311266,
1.4419442036986294,1.0521745333528625,0.12270057502622313,0.19592620955746853,2.793408623947534,0.7866897712879051,11.338654082864783,0.2900641551931054,0.3520364539372348,0.2085580407850846,
0.16272550978520736,0.10795093873732554,0.30903281913194697,0.10853422589607174,0.28330057489392657,0.16210539211702582,0.17931106060119276,0.04868677004998913,0.18893979265377084,0.12244119517496208,
0.13046883577294163,0.5880120513971442,0.05009171113085058,0.3868536140347565,1.023511045840697,0.8301379472609923,0.00581776436124472,0.08164428809427061,0.059278287522844746,0.07309202820436013,
0.011967083113500844,0.6653202097612048,0.0604579635030191,0.1843346284614876,1006.7579173576536,37982.62252893982,367657.7160786403]

Max variance Index:56

Max variance: 367657.7160786403

Percentage variance of max attripute: 90.40413231387978 %

In [1]:
val sparkVersion = "2.0.1"
val scalaVersion = scala.util.Properties.versionNumberString

sparkVersion: String = "2.0.1"
scalaVersion: String = "2.11.8"

In [3]:
classpath.add(
    "org.apache.spark" %% "spark-yarn" % sparkVersion,
    "org.apache.spark" %% "spark-mllib" % sparkVersion
)

0 new artifact(s)


In [10]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.SparkSession._
import org.apache.spark.sql.functions._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Row
import org.apache.spark.mllib.linalg.distributed.RowMatrix
import org.apache.spark.mllib.linalg.Matrix
import org.apache.spark.mllib.linalg.SingularValueDecomposition
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.mllib.feature.{StandardScaler, StandardScalerModel}
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Row
import org.apache.spark.sql.types.{StructField, StructType, LongType}
import org.apache.spark.sql.types.{DoubleType, IntegerType}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator


object classification {
    
    def main(args: Array[String]) {
        
        val sparkSession = SparkSession.builder
            .master("local")
            .appName("E1")
            .getOrCreate()
        
        val sc = sparkSession.sparkContext
        val sqlContext = new org.apache.spark.sql.SQLContext(sc)
        
        import sparkSession.implicits._
        
        val timing = System.nanoTime        
        
        // Load the data
        val text = sparkSession.sparkContext.textFile("files/spambase.data")
        // Separate into array
        val data = text.map(line => line.split(',').map(_.toDouble))
        // Organise data into feastures and labels
        val dataLP = data.map(t => (t(57), Vectors.dense(t.take(57))))
        
        val mat: RowMatrix = new RowMatrix(dataLP.values)
        val (pc, eigen) = mat.computePrincipalComponentsAndExplainedVariance(2)
        
        val projected: RowMatrix = mat.multiply(pc)
        
        val pcaRDD = projected.rows
        val pcaRDD2 = pcaRDD.map(_.toArray).map{case Array(pca1, pca2) => (pca1, pca2)}
        val pcaDF = sparkSession.createDataFrame(pcaRDD2).toDF("pca1", "pca2")
        
        val dataLP_label = dataLP.keys
        val DataLP_label = dataLP_label.toDF("label")
        
        val allrows = pcaDF.rdd.zip(DataLP_label.rdd).map{
            case (rowLeft, rowRight) => Row.fromSeq(rowLeft.toSeq ++ rowRight.toSeq)}
        
        val schema = StructType(pcaDF.schema.fields ++ DataLP_label.schema.fields)
        
        val the_DF: DataFrame = sqlContext.createDataFrame(allrows, schema)
        
        val DFF = the_DF.select(the_DF("label").cast(DoubleType).as("label"),
                                the_DF("pca1").cast(DoubleType).as("pca1"),
                                the_DF("pca2").cast(DoubleType).as("pca2"))
        
        
        
        // Index the labels
        val labelIndexer = new StringIndexer()
          .setInputCol("label")
          .setOutputCol("indexedLabel")
          .fit(DFF)
        
        val assembler = new VectorAssembler().setInputCols(Array("pca1", "pca2"))
                                            .setOutputCol("features")
        
       
    
       val lr = new LogisticRegression()
          .setMaxIter(5)
          .setElasticNetParam(0.1)
          .setRegParam(0.01)
        
        //Split the data into training and test sets (30% held out for testing).
        val Array(trainingData, testData) = DFF.randomSplit(Array(0.7, 0.3))



        // Pipeline.
        // Set the Pipeline and the Stages
        val pipeline = new Pipeline()
          .setStages(Array(labelIndexer, assembler, lr))

        // Train model.
        val model = pipeline.fit(trainingData)

        val predictions = model.transform(testData)

        // Select (prediction, true label) and compute test error.
        val evaluator = new MulticlassClassificationEvaluator()
          .setLabelCol("label")
          .setPredictionCol("prediction")
          .setMetricName("accuracy")
        val accuracy = evaluator.evaluate(predictions)
        println("Accuracy: "  + (accuracy))
        
        var i=0
        var totalVariance = 0.999999999
        var array = eigen.toArray
        var currentVariance = 0.0
        while (i < array.length){
        currentVariance += array(i)
        i += 1}
        println("variance: " + (currentVariance / totalVariance * 100))
        
        println("Time: " + (System.nanoTime-timing) / 1e9d)
}
}
classification.main(Array())

Accuracy: 0.6610909090909091
variance: 99.80699856872896
Time: 5.729179775


import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.SparkSession._
import org.apache.spark.sql.functions._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Row
import org.apache.spark.mllib.linalg.distributed.RowMatrix
import org.apache.spark.mllib.linalg.Matrix
import org.apache.spark.mllib.linalg.SingularValueDecomposition
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.mllib.feature.{StandardScaler, StandardScalerModel}
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Row
import org.apache.spark.sql.types.{StructField, StructType, LongType}
import org.apache.spark.sql.types.{DoubleType, IntegerType}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.evaluation.Multi

In [11]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.SparkSession._
import org.apache.spark.sql.functions._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Row
import org.apache.spark.mllib.linalg.distributed.RowMatrix
import org.apache.spark.mllib.linalg.Matrix
import org.apache.spark.mllib.linalg.SingularValueDecomposition
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.mllib.feature.{StandardScaler, StandardScalerModel}
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Row
import org.apache.spark.sql.types.{StructField, StructType, LongType}
import org.apache.spark.sql.types.{DoubleType, IntegerType}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

object classification2 {
    
    def main(args: Array[String]) {
        
        val sparkSession = SparkSession.builder
            .master("local")
            .appName("E1")
            .getOrCreate()
        
        val sc = sparkSession.sparkContext
        val sqlContext = new org.apache.spark.sql.SQLContext(sc)
        
        import sparkSession.implicits._
        
        val timing = System.nanoTime        
        
        // Load the data
        val text = sparkSession.sparkContext.textFile("files/spambase.data")
        // Separate into array
        val data = text.map(line => line.split(',').map(_.toDouble))
        // Organise data into feastures and labels
        val dataLP = data.map(t => (t(57), Vectors.dense(t.take(57))))
        
        
        
        val scaler = new StandardScaler(withMean = true, withStd = false).fit(dataLP.values).transform(dataLP.values)
        val mat2: RowMatrix = new RowMatrix(scaler)
        val svd: SingularValueDecomposition[RowMatrix, Matrix] = mat2.computeSVD(2, computeU = true)
        val singular: Vector = svd.s
        val pca: Matrix = svd.V
        val normalise = Vectors.dense(singular.toArray.map(a => a*a))
       
        
        val projected: RowMatrix = mat2.multiply(pca)
       
        
        
        val pcaRDD = projected.rows
        val pcaRDD2 = pcaRDD.map(_.toArray).map{case Array(pca1, pca2) => (pca1, pca2)}

        val pcaDF = sparkSession.createDataFrame(pcaRDD2).toDF("pca1", "pca2")

        val dataLP_label = dataLP.keys
        val DataLP_label = dataLP_label.toDF("label")
        
        val allrows = pcaDF.rdd.zip(DataLP_label.rdd).map{
            case (rowLeft, rowRight) => Row.fromSeq(rowLeft.toSeq ++ rowRight.toSeq)}
        
        val schema = StructType(pcaDF.schema.fields ++ DataLP_label.schema.fields)
        
        val the_DF: DataFrame = sqlContext.createDataFrame(allrows, schema)
        
        val DFF = the_DF.select(the_DF("label").cast(DoubleType).as("label"),
                                the_DF("pca1").cast(DoubleType).as("pca1"),
                                the_DF("pca2").cast(DoubleType).as("pca2"))
        
        
        
        // Index the labels
        val labelIndexer = new StringIndexer()
          .setInputCol("label")
          .setOutputCol("indexedLabel")
          .fit(DFF)
        
        val assembler = new VectorAssembler().setInputCols(Array("pca1", "pca2"))
                                            .setOutputCol("features")
        
       
    
       val lr = new LogisticRegression()
          .setMaxIter(5)
          .setElasticNetParam(0.1)
          .setRegParam(0.01)
        
        //Split the data into training and test sets (30% held out for testing).
        val Array(trainingData, testData) = DFF.randomSplit(Array(0.7, 0.3))



        // Pipeline.
        // Set the Pipeline and the Stages
        val pipeline = new Pipeline()
          .setStages(Array(labelIndexer, assembler, lr))

        // Train model.
        val model = pipeline.fit(trainingData)

        val predictions = model.transform(testData)

        // Select (prediction, true label) and compute test error.
        val evaluator = new MulticlassClassificationEvaluator()
          .setLabelCol("label")
          .setPredictionCol("prediction")
          .setMetricName("accuracy")
        val accuracy = evaluator.evaluate(predictions)
        println("Accuracy: "  + (accuracy))      
        
        println("Time: " + (System.nanoTime-timing) / 1e9d)
}
}
classification2.main(Array())

Accuracy: 0.6977904490377762
Time: 4.058148963


import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.SparkSession._
import org.apache.spark.sql.functions._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Row
import org.apache.spark.mllib.linalg.distributed.RowMatrix
import org.apache.spark.mllib.linalg.Matrix
import org.apache.spark.mllib.linalg.SingularValueDecomposition
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.mllib.feature.{StandardScaler, StandardScalerModel}
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Row
import org.apache.spark.sql.types.{StructField, StructType, LongType}
import org.apache.spark.sql.types.{DoubleType, IntegerType}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.evaluation.Multi

In [12]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.SparkSession._
import org.apache.spark.sql.functions._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Row
import org.apache.spark.mllib.linalg.distributed.RowMatrix
import org.apache.spark.mllib.linalg.Matrix
import org.apache.spark.mllib.linalg.SingularValueDecomposition
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.mllib.feature.{StandardScaler, StandardScalerModel}
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Row
import org.apache.spark.sql.types.{StructField, StructType, LongType}
import org.apache.spark.sql.types.{DoubleType, IntegerType}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.mllib.stat.{MultivariateStatisticalSummary, Statistics}

object variance {
    
    def main(args: Array[String]) {
        
        val sparkSession = SparkSession.builder
            .master("local")
            .appName("E1")
            .getOrCreate()
        
        val sc = sparkSession.sparkContext
        val sqlContext = new org.apache.spark.sql.SQLContext(sc)
        
        import sparkSession.implicits._
        
        val timing = System.nanoTime        
        
        // Load the data
        val text = sparkSession.sparkContext.textFile("files/spambase.data")
        // Separate into array
        val data = text.map(line => line.split(',').map(_.toDouble))
        // Organise data into feastures and labels
        val dataLP = data.map(t => (t(57), Vectors.dense(t.take(57))))
        
        val dataLPsummary: MultivariateStatisticalSummary = Statistics.colStats(dataLP.values)
        
        val variance_Array = dataLPsummary.variance.toArray
        
        print(dataLPsummary.variance)
        
        val maximumIndex = variance_Array.zipWithIndex.maxBy(_._1)._2
        
        println("'n")
        println("Max variance Index:" + maximumIndex)
        
        
        var maximumVariance = variance_Array.reduceLeft(_ max _)
        println("Max variance: " +maximumVariance)
        
        var totalVar = 0.0
        var counter = 0
        while (counter < variance_Array.length){
            totalVar += variance_Array(counter)
            counter +=1
        }
       
        println("Percentage variance of max attripute: " + maximumVariance / totalVar *100)
        
        println("Time: " + (System.nanoTime-timing) / 1e9d)
}
}
variance.main(Array())

[0.09324324068491728,1.6655843234835528,0.2541600473337551,1.946447346588171,0.45227342485092964,0.07497962843616227,0.15322633420900933,0.16085830998932185,0.0776267998072253,0.4157095251221379,0.040620251608818554,0.742524255407614,0.09062255504001963,0.11234819972973736,0.06699993229260176,0.6819319336533649,0.19718513519745243,0.28209102715855694,3.1523315909584833,0.25986228107311266,1.4419442036986294,1.0521745333528625,0.12270057502622313,0.19592620955746853,2.793408623947534,0.7866897712879051,11.338654082864783,0.2900641551931054,0.3520364539372348,0.2085580407850846,0.16272550978520736,0.10795093873732554,0.30903281913194697,0.10853422589607174,0.28330057489392657,0.16210539211702582,0.17931106060119276,0.04868677004998913,0.18893979265377084,0.12244119517496208,0.13046883577294163,0.5880120513971442,0.05009171113085058,0.3868536140347565,1.023511045840697,0.8301379472609923,0.00581776436124472,0.08164428809427061,0.059278287522844746,0.07309202820436013,0.011967083113500844,

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.SparkSession._
import org.apache.spark.sql.functions._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Row
import org.apache.spark.mllib.linalg.distributed.RowMatrix
import org.apache.spark.mllib.linalg.Matrix
import org.apache.spark.mllib.linalg.SingularValueDecomposition
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.mllib.feature.{StandardScaler, StandardScalerModel}
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Row
import org.apache.spark.sql.types.{StructField, StructType, LongType}
import org.apache.spark.sql.types.{DoubleType, IntegerType}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.evaluation.Multi